# COVID-19 Prediction

### Import necessary libraries

In [148]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [149]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [150]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from tqdm.auto import tqdm
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import albumentations
import cv2
import os
from typing import Dict, Tuple, List
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold

In [151]:
class config:
    batch_size = 16
    num_workers = 1
    epochs = 200
    hidden_units = 20

#### Utilizing GPUs

In [152]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [153]:
df_non_covid = pd.read_csv('/kaggle/input/covid19-china/NonCOVID-CT-MetaInfo.csv')
df_covid = pd.read_csv('/kaggle/input/covid19-china/COVID-CT-MetaInfo.csv')

In [154]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   File name        349 non-null    object
 1   Patient ID       349 non-null    object
 2   Age              169 non-null    object
 3   Gender           137 non-null    object
 4   Location         228 non-null    object
 5   Medical history  44 non-null     object
 6   Time             134 non-null    object
 7   Severity         178 non-null    object
 8   Other diseases   19 non-null     object
 9   DOI              203 non-null    object
 10  Captions         337 non-null    object
dtypes: object(11)
memory usage: 30.2+ KB


In [155]:
df_non_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  397 non-null    int64 
 1   image name  397 non-null    object
 2   patient id  397 non-null    object
dtypes: int64(1), object(2)
memory usage: 9.4+ KB


### Data transformation and augmentation

In [156]:
data_dir = '/kaggle/input/covid19-china'

In [157]:
train_transformer = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop((256), scale=(0.5,1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

val_transformer = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

In [158]:
data = datasets.ImageFolder(root=data_dir, # target folder of images
                                  transform=val_transformer, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)

print(f"Train data:\n{data}")

Train data:
Dataset ImageFolder
    Number of datapoints: 746
    Root location: /kaggle/input/covid19-china
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
               CenterCrop(size=(256, 256))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [159]:
# class_names = train_data.classes
# class_dict = train_data.class_to_idx

# img, label = train_data[0][0], train_data[0][1]

# img_permute = img.permute(1, 2, 0)

### Train test split

In [160]:
train_percent = 0.9

train_size = int(train_percent * len(data))
test_size = len(data) - train_size

train_dataset, test_dataset = random_split(data, [train_size, test_size])

train_dataset.dataset = datasets.ImageFolder(
    root=train_dataset.dataset.root,
    transform=train_transformer,
    target_transform=train_dataset.dataset.target_transform
)

train_dataset.dataset, test_dataset.dataset

(Dataset ImageFolder
     Number of datapoints: 746
     Root location: /kaggle/input/covid19-china
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
                RandomResizedCrop(size=(256, 256), scale=(0.5, 1), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=warn)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ),
 Dataset ImageFolder
     Number of datapoints: 746
     Root location: /kaggle/input/covid19-china
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
                CenterCrop(size=(256, 256))
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ))

In [161]:
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, num_workers=config.num_workers, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size, num_workers=config.num_workers, shuffle=False)

print(f"Number of samples in training set: {len(train_dataset)}")
print(f"Number of samples in testing set: {len(test_dataset)}")

Number of samples in training set: 671
Number of samples in testing set: 75


In [162]:
img, label = next(iter(train_loader))

print(img.shape, label.shape)

torch.Size([16, 3, 256, 256]) torch.Size([16])


In [172]:
class ConvModel(nn.Module):
    def __init__(self, input_shape, output_shape, hidden_units):
        super().__init__()
        
        self.block_1 = nn.Sequential(
                                nn.Conv2d(in_channels = input_shape,
                                         out_channels = hidden_units,
                                         kernel_size = 3,
                                         stride = 1,
                                         padding = 1),
                                nn.ReLU(),
                                nn.Conv2d(in_channels = hidden_units,
                                        out_channels = hidden_units,
                                        kernel_size = 3,
                                        stride = 1,
                                        padding = 1),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size = 2,
                                            stride = 2),
#                                 nn.BatchNorm2d()
        )
        
        self.block_2 = nn.Sequential(
                                nn.Conv2d(hidden_units, hidden_units, 3, padding = 1),
                                nn.ReLU(),
                                nn.Conv2d(hidden_units, hidden_units, 3, padding = 1),
                                nn.ReLU(),
                                nn.MaxPool2d(2),
#                                 nn.BatchNorm2d()
        )
        
        self.block_3 = nn.Sequential(
                                nn.Conv2d(hidden_units, hidden_units, 3, padding = 1),
                                nn.ReLU(),
                                nn.Conv2d(hidden_units, hidden_units, 3, padding = 1),
                                nn.ReLU(),
                                nn.MaxPool2d(2),
#                                 nn.BatchNorm2d()
        )
        
        self.block_4 = nn.Sequential(
                                nn.Conv2d(hidden_units, hidden_units, 3, padding = 1),
                                nn.ReLU(),
                                nn.Conv2d(hidden_units, hidden_units, 3, padding = 1),
                                nn.ReLU(),
                                nn.MaxPool2d(2),
#                                 nn.BatchNorm2d()
        )
        
        self.classifier = nn.Sequential(
                                nn.Flatten(),
                                nn.Linear(in_features = 5120, out_features = output_shape))
        
        
    def forward(self, x):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.block_3(x)
        x = self.block_4(x)
#         print(x.shape)
        x = self.classifier(x)
        return x

In [173]:
model = ConvModel(input_shape=3, hidden_units=config.hidden_units, output_shape = 1)
model.to(device)

ConvModel(
  (block_1): Sequential(
    (0): Conv2d(3, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_3): Sequential(
    (0): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_4): Sequential(
    (0): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (

In [174]:
print(model)

ConvModel(
  (block_1): Sequential(
    (0): Conv2d(3, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_3): Sequential(
    (0): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_4): Sequential(
    (0): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (

In [175]:
# model = ConvModel(input_shape=3, hidden_units=10, output_shape = 1)

# model_path = '/kaggle/input/cnn-models/model_cnn_proj_version_2.pt'
# model.load_state_dict(torch.load(f=model_path))

In [176]:
def train_step(model: torch.nn.Module,
              train_loader: torch.utils.data.DataLoader,
              device,
              loss_fn: torch.nn.Module,
              optimizer: torch.optim.Optimizer,
              train_loss):

    for batch, (X,y) in enumerate(train_loader):

        X, y = X.to(device), y.to(device)
        model.train()
        y_logits = model(X).squeeze()
        y_pred = torch.round(torch.sigmoid(y_logits))
        loss = loss_fn(y_pred, y.float())
        train_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return train_loss

In [177]:
def val_step(model: torch.nn.Module,
            val_loader: torch.utils.data.DataLoader,
            device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.inference_mode():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).squeeze()
            preds = torch.round(torch.sigmoid(outputs))
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
    return all_labels, all_preds


In [178]:
def train(model, data, train_dataset, epochs, device):

    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    for fold, (train_indices, val_indices) in enumerate(skf.split(train_dataset.dataset.imgs, train_dataset.dataset.targets), start=1):
        print(f"Fold {fold}")
        train_dataset = torch.utils.data.Subset(data, train_indices)
        val_dataset = torch.utils.data.Subset(data, val_indices)

        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

        torch.manual_seed(42)
        epochs = epochs

        for epoch in tqdm(range(epochs)):
    #         print(f'Epoch: {epoch} \n----------------')
            train_loss = 0
            train_step(model, train_loader, device, loss_fn, optimizer, train_loss)

        train_loss /= len(train_loader)

        all_labels, all_preds = val_step(model, val_loader, device)

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds)
        recall = recall_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds)

        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1 = np.mean(f1_scores)

    print(f'Average Accuracy: {avg_accuracy}')
    print(f"Average Precision: {avg_precision}")
    print(f"Average Recall: {avg_recall}")
    print(f"Average F1 Score: {avg_f1}")
    
    return model

In [179]:
def test(model, test_loader, device):
    test_pred = []
    test_labels = []
    model.eval()
    with torch.inference_mode():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).squeeze()
            preds = torch.round(torch.sigmoid(outputs))
            test_pred.extend(preds.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(test_labels, test_pred)
    precision = precision_score(test_labels, test_pred)
    recall = recall_score(test_labels, test_pred)
    f1 = f1_score(test_labels, test_pred)

    print(accuracy)
    print(precision)
    print(recall)
    print(f1)

In [180]:
print('Training the model')
train(model, data, train_dataset, config.epochs, device)
print('testing the model')
test(model, test_loader, device)

Training the model
Fold 1


  0%|          | 0/200 [00:00<?, ?it/s]

Fold 2


  0%|          | 0/200 [00:00<?, ?it/s]

Fold 3


...

Fold 4


  0%|          | 0/200 [00:00<?, ?it/s]

Fold 5


  0%|          | 0/200 [00:00<?, ?it/s]

Average Accuracy: 0.5321700223713647
Average Precision: 0.5321700223713647
Average Recall: 1.0
Average F1 Score: 0.6946578997331948
testing the model
0.6133333333333333
0.6133333333333333
1.0
0.7603305785123966


In [ ]:
torch.save(model.state_dict(), f'cnn-custom-model-{config.epochs}-{config.hidden_units}.pt')

In [ ]:
# model.to(device)
# print('testing the model')
# test(model, test_loader, device)

In [ ]:
# import torchvision.models as models

# model_resnet = models.resnet50(pretrained=True)

# for param in model_resnet.parameters():
#     param.requires_grad = False

# # Set the manual seeds
# torch.manual_seed(42)
# torch.cuda.manual_seed(42)

# # Replace the classifier with a new one
# model_resnet.fc = nn.Sequential(
#     nn.Linear(2048, 512),  # Change the input size as per your model
#     nn.ReLU(),
#     nn.Dropout(0.5),  # Dropout layer to reduce overfitting
#     nn.Linear(512, 1)  # Replace 'num_classes' with the number of your output classes
# )

# model_resnet.to(device);

In [ ]:
# from torchinfo import summary

# summary(model=model_resnet, 
#         input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
#         # col_names=["input_size"], # uncomment for smaller output
#         col_names=["input_size", "output_size", "num_params", "trainable"],
#         col_width=20,
#         row_settings=["var_names"]
# )

In [ ]:
# # # Do a summary *after* freezing the features and changing the output classifier layer (uncomment for actual output)
# summary(model_resnet, 
#         input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape" (batch_size, color_channels, height, width)
#         verbose=0,
#         col_names=["input_size", "output_size", "num_params", "trainable"],
#         col_width=20,
#         row_settings=["var_names"]
# )

In [ ]:
# print('Training the model')
# train(model_resnet, data, train_dataset, config.epochs, device)
# print('testing the model')
# test(model_resnet, test_loader, device)

In [ ]:
# accuracies, precisions, recalls, f1_scores

In [ ]:
# torch.load(model_resnet.state_dict(), 'cnn-resnet-version-3.pt')

In [ ]:
# # Import/install Gradio 
# try:
#     import gradio as gr
# except: 
#     !pip -q install gradio
#     import gradio as gr
    
# print(f"Gradio version: {gr.__version__}")

In [ ]:
from typing import Tuple, Dict

def predict(img):
    """Transforms and performs a prediction on img and returns prediction and time taken.
    """
    class_names = ['Covid']
    # Transform the target image and add a batch dimension
    img = val_transformer(img).unsqueeze(0)
    img = img.to(device)
    
    # Put model into evaluation mode and turn on inference mode
    model.eval()
    with torch.inference_mode():
        # Pass the transformed image through the model and turn the prediction logits into prediction probabilities
        pred_prob = torch.sigmoid(model(img))
        
    pred_probs = {'Covid' : float(pred_prob), 'Non Covid' : (1-float(pred_prob))}

    # Return the prediction dictionary and prediction time 
    return pred_probs

In [ ]:
import random
from PIL import Image
import numpy as np

# Randomly select a test image path
random_image_path = '/kaggle/input/covidct/CT_COVID/2020.02.10.20021584-p6-52%0.png'

# Open the target image
image = Image.open(random_image_path)
print(f"[INFO] Predicting on image at path: {random_image_path}\n")

# Predict on the target image and print out the outputs
pred_dict = predict(image)
print(f"Prediction label and probability dictionary: \n{pred_dict}")

In [ ]:
# np.array(image)

In [ ]:
# # Create a list of example inputs to our Gradio demo
# example_list = ['/kaggle/input/covid19-china/CT_NonCOVID/CT_NonCOVID/10%2.jpg',
#                '/kaggle/input/covid19-china/CT_NonCOVID/CT_NonCOVID/1030.png',
#                '/kaggle/input/covid19-china/CT_NonCOVID/CT_NonCOVID/1029.png']
# example_list

In [ ]:
# def deploy():
#     # Create title, description and article strings
#     title = "Corona Prediction"
#     description = "A Convolutional Neural Network To classify whether a person have Corona or not using CT  Scans."
#     article = "Created by Thenujan Nagaratnam for DNN module at UoM"

#     # Create the Gradio demo
#     demo = gr.Interface(fn=predict, # mapping function from input to output
#                         inputs=gr.Image(type="pil"), # what are the inputs?
#                         outputs=[gr.Label(num_top_classes=2, label="Predictions")], # our fn has two outputs, therefore we have two outputs
#                         examples=example_list, 
#                         title=title,
#                         description=description,
#                         article=article)

#     # Launch the demo!
#     demo.launch(debug=True, # print errors locally?
#                 share=True) # generate a publically shareable URL?

In [ ]:
# deploy()

In [2]:
pip install gradio_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.4/298.4 kB 6.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
from gradio_client import Client
import json

client = Client("https://thenujan-covid-cnn.hf.space/")
result = client.predict(
                '/kaggle/input/covid19-china/CT_NonCOVID/CT_NonCOVID/10%2.jpg',	# str (filepath or URL to image) in 'img' Image component
                api_name="/predict"
)

with open(result, 'r') as json_file:
    data = json.load(json_file)

print(data)

Loaded as API: https://thenujan-covid-cnn.hf.space/ ✔
{'label': 'Covid', 'confidences': [{'label': 'Covid', 'confidence': 0.500697672367096}, {'label': 'Non Covid', 'confidence': 0.49930232763290405}]}


In [5]:
# from gradio_client import Client

# client = Client("https://thenujan-vpr-deploy.hf.space/")
# result = client.predict(
# 				"https://raw.githubusercontent.com/gradio-app/gradio/main/test/test_files/bus.png",	# str (filepath or URL to image) in 'image' Image component
# 				api_name="/predict"
# )
# print(result)

Loaded as API: https://thenujan-vpr-deploy.hf.space/ ✔
/tmp/gradio/tmpcvmyph02.json


In [9]:
# with open(result, 'r') as json_file:
#     data = json.load(json_file)

# print(data)

In [10]:
# data['similar_image_ids'][0]

In [ ]:
# pip install open-clip-torch

In [ ]:
# import open_clip

# model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:Thenujan/ViT-H-14')
# tokenizer = open_clip.get_tokenizer('hf-hub:Thenujan/ViT-H-14')

In [ ]:
model

In [ ]:
# import torch as th
# backbone = open_clip.create_model_and_transforms('ViT-H-14', None)[0].visual
# backbone.load_state_dict(th.load('https://huggingface.co/Thenujan/ViT-H-14/main/model.bin'))  #https://huggingface.co/hca97/aicrowd-visual-recognition-models/blob/main/model1.pt
# # backbone, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:Thenujan/ViT-H-14')
# backbone.eval()   # Dropping unecessary layers

In [ ]:
# import open_clip

# model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:Thenujan/ViT-H-14')
# tokenizer = open_clip.get_tokenizer('hf-hub:Thenujan/ViT-H-14')

In [ ]:
# costom model with 3 layers, 60 epochs, hidden units 10
#     Average Accuracy: 0.5549709172259508
#     Average Precision: 0.5552134116025019
#     Average Recall: 0.8262974683544304
#     Average F1 Score: 0.6637566887530646
#     testing the model
#     0.5333333333333333
#     0.5333333333333333
#     0.8205128205128205
#     0.6464646464646464

# costom model with 3 layers, 200 epochs, hidden units 10
#     Average Accuracy: 0.5321700223713647
#     Average Precision: 0.5321700223713647
#     Average Recall: 1.0
#     Average F1 Score: 0.6946578997331948
#     testing the model
#     0.5333333333333333
#     0.5333333333333333
#     1.0
#     0.6956521739130436

# custom model with 5 layers, 200 epochs, hidden units 10
#     Average Accuracy: 0.5321700223713647
#     Average Precision: 0.5321700223713647
#     Average Recall: 1.0
#     Average F1 Score: 0.6946578997331948
#     testing the model
#     0.6133333333333333
#     0.6133333333333333
#     1.0
#     0.7603305785123966

# custom model with 5 layers, 200 epochs, hidden units 20
#     Average Accuracy: 0.5321700223713647
#     Average Precision: 0.5321700223713647
#     Average Recall: 1.0
#     Average F1 Score: 0.6946578997331948
#     testing the model
#     0.6133333333333333
#     0.6133333333333333
#     1.0
#     0.7603305785123966

In [184]:
# pip install open-clip-torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# import open_clip

# model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:laion/CLIP-ViT-H-14-laion2B-s32B-b79K')
# tokenizer = open_clip.get_tokenizer('hf-hub:laion/CLIP-ViT-H-14-laion2B-s32B-b79K')